In [1]:
import pandas as pd
import numpy as np
import spacy
import pickle
import random
import time
import re
# Import required modules

In [2]:
# make train data in spacy required format
summary = pd.read_csv("../1000_tagged_simanchala/tech_sort1k.csv")
tech = pd.read_csv("../Dataset/Technology Summary Matching Project - technologies.csv")

In [3]:
# sorting by id
summary = summary.sort_values(by=['id'], ascending=False)
summary.reset_index(inplace = True, drop = True)
summary.drop(columns = ["Note"], axis =1, inplace = True)

In [4]:
# tech.columns = ["index", "title"]
# tech.drop(columns = ["index"], axis =1 , inplace = True)
tech.reset_index(inplace = True, drop = True)

In [5]:
# removing garbage words from tech sheet
r = pd.read_csv("../Remove.csv")

In [6]:
for i in range(0,len(r)):
    r["Remove"][i] = r["Remove"][i].lower()

In [7]:
remove_list = r["Remove"].tolist()
remove_list.append("ve")

In [8]:
# tech_copy = tech.copy()

In [9]:
# tech_copy.set_index("title", inplace = True)

In [10]:
type(tech["title"])

pandas.core.series.Series

In [11]:
tech.to_csv("tech_post_garbage_drop.csv", index = False)

In [12]:
print("\n Summary Data \n", summary.head())
print("\n Technology Sheet Data \n", tech.head())


 Summary Data 
                             id exact_matched_patt_contextual  \
0  zsDSYc5UzpyXekOABZBfnQ_0000                           NaN   
1  zs1YNrrCeorFkGLEXSSp-A_0000                           NaN   
2  zowirq8ZhxhchApaRMhNSA_0000                           NaN   
3  zoQVkPCfXB9n3AaPGHHwzg_0000                           NaN   
4  zmIr2glBZ3Ef8CSS0jw1og_0000                           NaN   

                                           summaries  
0  senior director of clinical services housing a...  
1  i believe that passions are meant to be shared...  
2  bill bryant is founder and chairman of bryant ...  
3  undertaking a trilingual masters degree in eur...  
4  career objective a role within marketing and n...  

 Technology Sheet Data 
          title
0      Lattice
1  WorkflowMax
2     InGenius
3      DealHub
4        Ebsta


In [13]:
print(len(tech), len(summary))

29348 1001


In [14]:
summary = summary.iloc[0:2]

Getting indexes for matching companies in the form (start_index, end_index, "ORG")

In [15]:
tech_list = []
summary_list = []
id_list = []

In [16]:
def KMP_String(pattern, text):
    a = len(text)
    b = len(pattern)

    prefix_arr = get_prefix_arr(pattern, b)

  
    initial_point = []

    m = 0
    n = 0

  
    while m != a:
       
        if text[m] == pattern[n]:
            m += 1
            n += 1
      
        else:
            n = prefix_arr[n-1]

       
        if n == b:
            initial_point.append(m-n)
            n = prefix_arr[n-1]
        elif n == 0:
            m += 1
            
    return initial_point



def get_prefix_arr(pattern, b):
    prefix_arr = [0] * b
    n = 0
    m = 1

    while m != b:
        if pattern[m] == pattern[n]:
            n += 1
            prefix_arr[m] = n
            m += 1
        elif n != 0:
                n = prefix_arr[n-1]
        else:
            prefix_arr[m] = 0
            m += 1

    return prefix_arr

In [17]:
# converting string of list data that is being dumped from source here to list properly to iterate easily later on

def dump_format_change(df, col_name):
    
    for i in range(0, len(df)):
        print(i)
        if df[col_name][i] is not np.nan:
            demo = df[col_name][i]
            res = demo.strip('][')
            resl = []
            resl.append(res)
            df[col_name][i] = resl
        
        else:
            df[col_name][i] = '[]'
            demo = df[col_name][i]
            res = demo.strip('][')
            resl = []
            resl.append(res)
            df[col_name][i] = resl
    return df 

In [18]:
summary = dump_format_change(summary, "summaries")

0
1


In [19]:
# trying joblib for paraller processing
#Import package
from joblib import Parallel, delayed
from joblib import Memory

In [20]:
backend = 'multiprocessing'

In [21]:
def final_run_string(summary_copy, tech):
    
    tech_list = []
    summary_list = []
    id_list = []
    spacy_format_list = []
    
    start = time.time()

    for i in range(0, len(summary_copy)):
        print(i)
        string = summary_copy.loc[i, "summaries"][0]
        string = string.lower()

        for j in range(0, len(tech)):
            pat = tech.loc[j,"title"]
            pat = pat.lower()


            initial_index = KMP_String(pat, string)
            
            if len(initial_index)!=0:
                print("\n summary index : \n", i, "\n technology index: \n", j,"\n initial index: \n", initial_index)

                tech_list.append(pat)    
                summary_list.append(string)
                id_list.append(summary_copy.loc[i,"id"])
                
                for k in range(0, len(initial_index)):
                    spacy_format_list.append((summary_copy.loc[i,"id"],initial_index[k],initial_index[k]+len(pat)-1, "ORG",pat))
                    
    end = time.time() - start
    print("\n Time Taken \n", end)
    return tech_list, summary_list, id_list, spacy_format_list

In [22]:
tech_list, summary_list, id_list, spacy_format_list = final_run_string(summary, tech)

0

 summary index : 
 0 
 technology index: 
 1497 
 initial index: 
 [106, 267, 1154]

 summary index : 
 0 
 technology index: 
 2083 
 initial index: 
 [135, 560, 904, 1021, 1448, 1792]

 summary index : 
 0 
 technology index: 
 4648 
 initial index: 
 [794, 1682]

 summary index : 
 0 
 technology index: 
 5310 
 initial index: 
 [223, 1110]

 summary index : 
 0 
 technology index: 
 5384 
 initial index: 
 [142, 1028]

 summary index : 
 0 
 technology index: 
 5451 
 initial index: 
 [696, 1584]

 summary index : 
 0 
 technology index: 
 6286 
 initial index: 
 [316, 1204]

 summary index : 
 0 
 technology index: 
 6506 
 initial index: 
 [27, 68, 940, 989, 1828]

 summary index : 
 0 
 technology index: 
 6878 
 initial index: 
 [120, 870, 1006, 1758]

 summary index : 
 0 
 technology index: 
 7020 
 initial index: 
 [12, 706, 925, 1594, 1813, 1888]

 summary index : 
 0 
 technology index: 
 7575 
 initial index: 
 [129, 879, 1015, 1767]

 summary index : 
 0 
 technology 


 Time Taken 
 39.71403980255127


In [23]:
tech["title"][527]

'Grow'

In [24]:
len(tech["title"][527])

4

In [25]:
746+len(tech["title"][527])-1
# end_index = start_index +len(company name)-1

749

In [26]:
print(len(tech_list), len(summary_list), len(id_list), len(spacy_format_list))

81 81 81 478


In [27]:
spacy_format_list

[('zsDSYc5UzpyXekOABZBfnQ_0000', 106, 111, 'ORG', 'vision'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 267, 272, 'ORG', 'vision'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 1154, 1159, 'ORG', 'vision'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 135, 138, 'ORG', 'rain'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 560, 563, 'ORG', 'rain'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 904, 907, 'ORG', 'rain'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 1021, 1024, 'ORG', 'rain'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 1448, 1451, 'ORG', 'rain'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 1792, 1795, 'ORG', 'rain'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 794, 796, 'ORG', 'ora'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 1682, 1684, 'ORG', 'ora'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 223, 226, 'ORG', 'alis'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 1110, 1113, 'ORG', 'alis'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 142, 147, 'ORG', 'and co'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 1028, 1033, 'ORG', 'and co'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000', 696, 703, 'ORG', 'tropical'),
 ('zsDSYc5UzpyXekOABZBfnQ_0000

In [28]:
tech_list_copy = tech_list
id_list_copy = id_list
summary_list_copy = summary_list

In [29]:
def getDuplicatesWithInfo(listOfElems):
    ''' Get duplicate element in a list along with thier indices in list
     and frequency count'''
    dictOfElems = dict()
    index = 0
    # Iterate over each element in list and keep track of index
    for elem in listOfElems:
        # If element exists in dict then keep its index in list & increment its frequency
        if elem in dictOfElems:
            dictOfElems[elem][0] += 1
            dictOfElems[elem][1].append(index)
        else:
            # Add a new entry in dictionary 
            dictOfElems[elem] = [1, [index]]
        index += 1    
 
    dictOfElems = { key:value for key, value in dictOfElems.items() if value[0] > 1}
    return dictOfElems

In [30]:
# List of strings
listOfElems = id_list
dictOfElems = getDuplicatesWithInfo(listOfElems)

In [31]:
df = pd.DataFrame(list(zip(id_list, summary_list, tech_list)), 
               columns =['id', 'summaries', "Tech_from_string_match"])

In [32]:
df.head()

,id,summaries,Tech_from_string_match
0,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,vision
1,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,rain
2,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,ora
3,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,alis
4,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,and co


In [33]:
for key, value in dictOfElems.items():
#     print("******************")
#     print("\n ID is..", key )
    a = df["Tech_from_string_match"][value[1][0]] #this needs to be changed
    b = []
    b.insert(0,a)
#     print("List at first index", b)
    for i in range(0, len(value[1])-1):
        b.append(df["Tech_from_string_match"][value[1][i+1]])
        df["Tech_from_string_match"][value[1][0]] = b
        
    for i in range(0, len(value[1])-1):
        df = df.drop(index = [value[1][i+1]])
#         print('Element = ', key , ' :: Repeated Count = ', value[0] , ' :: Index Positions =  ', value[1])

In [34]:
df.reset_index(inplace = True, drop = True)

In [35]:
spacy_format_list[0][4]

'vision'

In [36]:
def spacy_format_align(unique_id, spacy_format_list):
    
    new_spacy_list = []
    for i in range(0, len(spacy_format_list)):
        if spacy_format_list[i][0] == unique_id:
            new_spacy_list.append([spacy_format_list[i][4],spacy_format_list[i][1], spacy_format_list[i][2], spacy_format_list[i][3]])
#             print(new_spacy_list)
                
    return new_spacy_list

In [37]:
df["spacy_format"] = df.apply(lambda x: spacy_format_align(x["id"], spacy_format_list), axis = 1)

In [38]:
len(df["spacy_format"][0])

239

In [39]:
df["spacy_format"][0]

[['vision', 106, 111, 'ORG'],
 ['vision', 267, 272, 'ORG'],
 ['vision', 1154, 1159, 'ORG'],
 ['rain', 135, 138, 'ORG'],
 ['rain', 560, 563, 'ORG'],
 ['rain', 904, 907, 'ORG'],
 ['rain', 1021, 1024, 'ORG'],
 ['rain', 1448, 1451, 'ORG'],
 ['rain', 1792, 1795, 'ORG'],
 ['ora', 794, 796, 'ORG'],
 ['ora', 1682, 1684, 'ORG'],
 ['alis', 223, 226, 'ORG'],
 ['alis', 1110, 1113, 'ORG'],
 ['and co', 142, 147, 'ORG'],
 ['and co', 1028, 1033, 'ORG'],
 ['tropical', 696, 703, 'ORG'],
 ['tropical', 1584, 1591, 'ORG'],
 ['manage', 316, 321, 'ORG'],
 ['manage', 1204, 1209, 'ORG'],
 ['vserv', 27, 31, 'ORG'],
 ['vserv', 68, 72, 'ORG'],
 ['vserv', 940, 944, 'ORG'],
 ['vserv', 989, 993, 'ORG'],
 ['vserv', 1828, 1832, 'ORG'],
 ['inform', 120, 125, 'ORG'],
 ['inform', 870, 875, 'ORG'],
 ['inform', 1006, 1011, 'ORG'],
 ['inform', 1758, 1763, 'ORG'],
 ['tor', 12, 14, 'ORG'],
 ['tor', 706, 708, 'ORG'],
 ['tor', 925, 927, 'ORG'],
 ['tor', 1594, 1596, 'ORG'],
 ['tor', 1813, 1815, 'ORG'],
 ['tor', 1888, 1890, 'ORG'

In [40]:
len(df["Tech_from_string_match"][0])

39

In [41]:
df["Tech_from_string_match"][0]

['vision',
 'rain',
 'ora',
 'alis',
 'and co',
 'tropical',
 'manage',
 'vserv',
 'inform',
 'tor',
 'care',
 'circle',
 'scons',
 'eservice',
 'emanage',
 'service',
 'ava',
 'staf',
 'c',
 'tia',
 'bob',
 'tint',
 'ion',
 'iri',
 'soci',
 'awareness',
 'octo',
 'ning',
 'storm',
 'sass',
 've',
 'h',
 'flo',
 'steer',
 'intellect',
 'cat',
 'vocate',
 'ntent',
 'intel']

In [42]:
df.head()

,id,summaries,Tech_from_string_match,spacy_format
0,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,"[vision, rain, ora, alis, and co, tropical, ma...","[[vision, 106, 111, ORG], [vision, 267, 272, O..."
1,zs1YNrrCeorFkGLEXSSp-A_0000,i believe that passions are meant to be shared...,"[grow, xperience, rain, ark, real, ora, alis, ...","[[grow, 746, 749, ORG], [xperience, 1513, 1521..."


In [43]:
# Applying Regex to filter out Invalid Words

def get_exact_match(txt, tech):
    try:
        patt = '|'.join(['\\b'+elem+'\\b' for elem in tech])
        matched_patt = re.findall(patt,txt)
    except Exception as e:
        print(e)
        matched_patt = []
    return matched_patt

In [44]:
df['exact_matched_patt'] = df.apply(lambda x: get_exact_match(x["summaries"], x["Tech_from_string_match"]), axis = 1)

In [45]:
# Applying Regex to filter out Invalid Words

def get_exact_match_2(txt, tech):
    try:
        patt = '|'.join(['\\s'+elem+'\\s' for elem in tech])
        matched_patt = re.findall(patt,txt[0])
    except Exception as e:
        print(e)
        matched_patt = []
    return matched_patt

In [46]:
df['matched_NER_final'] = df.apply(lambda x: get_exact_match_2(x["summaries"], x["exact_matched_patt"]), axis = 1)

In [47]:
from collections import OrderedDict 

In [48]:
for ind in df.index:
    res = list(OrderedDict.fromkeys(df["exact_matched_patt"][ind]))
    df["exact_matched_patt"][ind] = res

In [49]:
for i in range(0, len(df)):
    for word in list(df["exact_matched_patt"][i]):
        if word in remove_list:
            df["exact_matched_patt"][i].remove(word)

In [50]:
df.head()

,id,summaries,Tech_from_string_match,spacy_format,exact_matched_patt,matched_NER_final
0,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,"[vision, rain, ora, alis, and co, tropical, ma...","[[vision, 106, 111, ORG], [vision, 267, 272, O...","[care, awareness, tropical, storm]",[]
1,zs1YNrrCeorFkGLEXSSp-A_0000,i believe that passions are meant to be shared...,"[grow, xperience, rain, ark, real, ora, alis, ...","[[grow, 746, 749, ORG], [xperience, 1513, 1521...","[storyteller, brain, grow, able]",[]


In [51]:
# df["spacy_format"][0].remove(df["spacy_format"][0][0])

In [52]:
# for i in list(spacy_format):
#     print(i)
#     if i[0] not in final_tech:
#         spacy_format.remove(i)

In [53]:
# remove invalid entries from spacy format column

def remove_invalid(final_tech, spacy_format):
    
    for i in list(spacy_format):
        if i[0] not in final_tech:
            spacy_format.remove(i)

    return spacy_format          

In [54]:
df["spacy_format_final"] = df.apply(lambda x: remove_invalid(x["exact_matched_patt"], x["spacy_format"]), axis = 1)

In [55]:
df["spacy_format_final"][0]

[['tropical', 696, 703, 'ORG'],
 ['tropical', 1584, 1591, 'ORG'],
 ['care', 129, 132, 'ORG'],
 ['care', 879, 882, 'ORG'],
 ['care', 1015, 1018, 'ORG'],
 ['care', 1767, 1770, 'ORG'],
 ['awareness', 506, 514, 'ORG'],
 ['awareness', 1394, 1402, 'ORG'],
 ['storm', 705, 709, 'ORG'],
 ['storm', 1593, 1597, 'ORG']]

In [56]:
df.head()

,id,summaries,Tech_from_string_match,spacy_format,exact_matched_patt,matched_NER_final,spacy_format_final
0,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,"[vision, rain, ora, alis, and co, tropical, ma...","[[tropical, 696, 703, ORG], [tropical, 1584, 1...","[care, awareness, tropical, storm]",[],"[[tropical, 696, 703, ORG], [tropical, 1584, 1..."
1,zs1YNrrCeorFkGLEXSSp-A_0000,i believe that passions are meant to be shared...,"[grow, xperience, rain, ark, real, ora, alis, ...","[[grow, 746, 749, ORG], [brain, 409, 413, ORG]...","[storyteller, brain, grow, able]",[],"[[grow, 746, 749, ORG], [brain, 409, 413, ORG]..."


dumping spacy format data into json now

In [57]:
# for i in range(0, len(list(spacy_f))):
#     print(i)
#     name = spacy_f[i][0]
#     print(name)
#     (spacy_f[i]).remove(name)
# #     spacy_f.remove(i[0])

In [58]:
def remove_name(spacy_format_final):
    for i in range(0, len(list(spacy_format_final))):
        name = spacy_format_final[i][0]
        spacy_format_final[i].remove(name)
        
    return spacy_format_final

In [59]:
df["spacy_format_final"] = df.apply(lambda x: remove_name(x["spacy_format_final"]), axis = 1)

In [60]:
df.head()

,id,summaries,Tech_from_string_match,spacy_format,exact_matched_patt,matched_NER_final,spacy_format_final
0,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,"[vision, rain, ora, alis, and co, tropical, ma...","[[696, 703, ORG], [1584, 1591, ORG], [129, 132...","[care, awareness, tropical, storm]",[],"[[696, 703, ORG], [1584, 1591, ORG], [129, 132..."
1,zs1YNrrCeorFkGLEXSSp-A_0000,i believe that passions are meant to be shared...,"[grow, xperience, rain, ark, real, ora, alis, ...","[[746, 749, ORG], [409, 413, ORG], [424, 428, ...","[storyteller, brain, grow, able]",[],"[[746, 749, ORG], [409, 413, ORG], [424, 428, ..."


In [61]:
df["spacy_format_final"][0]

[[696, 703, 'ORG'],
 [1584, 1591, 'ORG'],
 [129, 132, 'ORG'],
 [879, 882, 'ORG'],
 [1015, 1018, 'ORG'],
 [1767, 1770, 'ORG'],
 [506, 514, 'ORG'],
 [1394, 1402, 'ORG'],
 [705, 709, 'ORG'],
 [1593, 1597, 'ORG']]

Format for training data will be ..
- (summary_text, 
    {entities: [
    (s_idx, e_idx, "ORG"),
    ]})

In [72]:
def format_arrange(summaries, spacy_format_final):
    
    d = {}
    for i in spacy_format_final:
        i = tuple(i)
        d.setdefault("entities", [])
        d["entities"].append(i)
    
    summary_tuple = (summaries, d)

    return summary_tuple

In [73]:
df["spacy_final"] = df.apply(lambda x: format_arrange(x["summaries"], x["spacy_format_final"]), axis = 1)

In [74]:
df.head()

,id,summaries,Tech_from_string_match,spacy_format,exact_matched_patt,matched_NER_final,spacy_format_final,spacy_final
0,zsDSYc5UzpyXekOABZBfnQ_0000,senior director of clinical services housing a...,"[vision, rain, ora, alis, and co, tropical, ma...","[[696, 703, ORG], [1584, 1591, ORG], [129, 132...","[care, awareness, tropical, storm]",[],"[[696, 703, ORG], [1584, 1591, ORG], [129, 132...",(senior director of clinical services housing ...
1,zs1YNrrCeorFkGLEXSSp-A_0000,i believe that passions are meant to be shared...,"[grow, xperience, rain, ark, real, ora, alis, ...","[[746, 749, ORG], [409, 413, ORG], [424, 428, ...","[storyteller, brain, grow, able]",[],"[[746, 749, ORG], [409, 413, ORG], [424, 428, ...",(i believe that passions are meant to be share...


In [75]:
df["spacy_final"][0]

('senior director of clinical services housing assistance and hiv aids services with focus in clinical supervision trauma informed care trainer and consultant program development and leadership building behavior support specialist behavior therapist lcsw/ provide supervision to therapist obtaining credentials former management of autism circles of treatment support grant consultation in the area of intellectual disabilities and mental health needs expertise in the area and advocate of social and sexual awareness and needs for individuals participated and trained as a volunteer for the kentucky crisis response team volunteered for american red cross to debrief victims of katrina, flood and tropical storm disasters presentations in a variety of areas to assist clients and staff in behavioral and mental health needs former member of steering committee for trauma informed care and development of training, senior director of clinical services\nhousing programs with focus in clinical service